In [ ]:
library(Rsubread)
library(repr)
library(ggplot2)
library(ggrepel)
require(dplyr)
library(tidyr) 
library(reshape2)
library("DESeq2")
library(RColorBrewer)
library( "gplots" )
require('biomaRt')
require('pheatmap')
require(GenomicFeatures)
require('plotly')
require(TxDb.Dmelanogaster.UCSC.dm6.ensGene)
require('edgeR')
library("ggpubr")
library('htmlwidgets')

In [ ]:
#import ENSEMBL database
listEnsembl()
mart <- useMart(biomart = "ensembl", 
                dataset = "dmelanogaster_gene_ensembl", 
                host = "https://www.ensembl.org")
mart <- useDataset(dataset="dmelanogaster_gene_ensembl", mart=mart)

# 1- FeatureCounts

In [ ]:
annotation_file="../../../../genome/Drosophila_melanogaster.BDGP6.32.107.gtf"
fasta_file="../../../../genome/Drosophila_melanogaster.BDGP6.32.dna_sm.toplevel.fa"


# FOR WT SAMPLES :
counts.matrix.g080 <-featureCounts(
              files = c('FBE14.bam','FBE15.bam','FBE16.bam','FBE17.bam','FBE18.bam','FBE19.bam'), 
              GTF.attrType = "gene_id",
              annot.ext = annotation_file,
              genome= fasta_file,
              isGTFAnnotationFile = TRUE,
              useMetaFeatures=TRUE,
              GTF.featureType= "gene",
              nthreads = 40,
              isPairedEnd = TRUE,
              requireBothEndsMapped = TRUE,
              countChimericFragments = FALSE,
              juncCounts = TRUE)



# FOR DPLD SAMPLES :
counts.matrix.dpld <-featureCounts(
              files = c('FBE21.bam','FBE22.bam','FBE23.bam','FBE24.bam','FBE25.bam','FBE26.bam'), 
              GTF.attrType = "gene_id",
              annot.ext = annotation_file,
              genome= fasta_file,
              isGTFAnnotationFile = TRUE,
              useMetaFeatures=TRUE,
              GTF.featureType= "gene",
              nthreads = 40,
              isPairedEnd = TRUE,
              requireBothEndsMapped = TRUE,
              countChimericFragments = FALSE,
              juncCounts = TRUE)

In [ ]:
counts.matrix.g080 = readRDS("../../Analysis/synaptosome-july-2022/4.feature-counts/noCnoB/counts_G080_flybase.rds")

In [ ]:
counts.matrix.dpld = readRDS("../../Analysis/synaptosome-july-2022/4.feature-counts/noCnoB/counts_dPLD_flybase.rds")

In [ ]:
#convert the matrix to data frame
count.g080=as.data.frame(counts.matrix.g080$counts)
count.dpld=as.data.frame(counts.matrix.dpld$counts)

## 2- Data preparation

## -- WT --

In [ ]:
print("original dimension")
dim(count.g080)

#here we remove all the Fbti
keep=NULL
keep <- count.g080[!grepl("FBti", row.names(count.g080)),] 
genesTokeep=which(rownames(count.g080) %in% row.names(keep))
count.g080 <- count.g080[genesTokeep, ]
print("after removing genes with FBti")
dim(count.g080)

#here we remove all the transposable element (they start with RR)
keep=NULL
keep <- count.g080[!grepl("RR", row.names(count.g080)),] 
genesTokeep=which(rownames(count.g080) %in% row.names(keep))
count.g080 <- count.g080[genesTokeep, ]
print("after removing genes with RR")
dim(count.g080)

#here we remove genes with 0 reads in all samples
keep_genes <- rowSums(count.g080) > 0 #TRUE if the gene has more that 0 reads, if not : FALSE
count.g080 <- count.g080[ keep_genes, ]
print("after removing genes with 0 reads")
dim(count.g080) #I removed the 2000 genes with 0 reads

#cpm filtering
keep <- rowSums(cpm(count.g080) > 0.5) >= 3
count.g080<- count.g080[keep , ]
print('after removing genes with cpm')
dim(count.g080)

## -- DPLD --

In [ ]:
print("original dimension")
dim(count.dpld)

#here we remove all the Fbti
keep=NULL
keep <- count.dpld[!grepl("FBti", row.names(count.dpld)),] 
genesTokeep=which(rownames(count.dpld) %in% row.names(keep))
count.dpld <- count.dpld[genesTokeep, ]
print("after removing genes with FBti")
dim(count.dpld)

#here we remove all the transposable element (they start with RR)
keep=NULL
keep <- count.dpld[!grepl("RR", row.names(count.dpld)),] 
genesTokeep=which(rownames(count.dpld) %in% row.names(keep))
count.dpld <- count.dpld[genesTokeep, ]
print("after removing genes with RR")
dim(count.dpld)

#here we remove genes with 0 reads in all samples
keep_genes <- rowSums(count.dpld) > 0 #TRUE if the gene has more that 0 reads, if not : FALSE
count.dpld <- count.dpld[ keep_genes, ]
print("after removing genes with 0 reads")
dim(count.dpld) #I removed the 2000 genes with 0 reads

#cpm filtering
keep <- rowSums(cpm(count.dpld) > 0.5) >= 3
count.dpld<- count.dpld[keep , ]
print('after removing genes with cpm')
dim(count.dpld)

# 3- Build DESeq2 matrix

The files are organized as follows:
FBE14 : original input
FBE15 : fraction from FBE14 enriched in synaptosomes
FBE16 : original input
FBE17 : fraction from FBE16 enriched in synaptosomes
...
so there is 2 relationships between samples to consider :
- condition : input or synaptosomes enriched
- sample.ID : (FBE14+FBE15), (FBE16+FB17), ...

In [ ]:
#setting up sample info:
sample_info <- DataFrame(condition = c("input","synap", "input", "synap", "input", "synap"),sample.ID=c("1","1","2","2","3","3"), row.names = names(count) )
sample_info$condition=as.factor(sample_info$condition)
sample_info$sample.ID=as.factor(sample_info$sample.ID)

## -- WT --

In [ ]:
#intializing dds object:
dds.g080 <- DESeqDataSetFromMatrix(countData = count.g080,colData = sample_info,design = ~ condition + sample.ID)


## -- DPLD --

In [ ]:
#intializing dds object:
dds.dpld <- DESeqDataSetFromMatrix(countData = count.dpld,colData = sample_info,design = ~ condition + sample.ID)


### plot 1 : number of reads per sample

In [ ]:
g=melt(colSums(counts(dds.g080)))
readCounts_plot<-ggplot(data=g, aes(x=rownames(g), y=value, fill=rownames(g))) +
  geom_bar(stat="identity") +
  #scale_fill_manual(values = mycolors)
    scale_fill_brewer(palette="Dark2") + theme(legend.position = "none") +
  geom_text(aes(label=value), vjust=1.6, color="white", size=6) +
  ggtitle("Total number of sequenced genes per sample")+ theme(text = element_text(size=20), axis.text.x = element_text(size=15, angle = 45))+
  xlab("Samples") + ylab("read counts")
readCounts_plot

In [ ]:
g=melt(colSums(counts(dds.dpld)))
readCounts_plot<-ggplot(data=g, aes(x=rownames(g), y=value, fill=rownames(g))) +
  geom_bar(stat="identity") +
  #scale_fill_manual(values = mycolors)
    scale_fill_brewer(palette="Dark2") + theme(legend.position = "none") +
  geom_text(aes(label=value), vjust=1.6, color="white", size=6) +
  ggtitle("Total number of sequenced genes per sample")+ theme(text = element_text(size=20), axis.text.x = element_text(size=15, angle = 45))+
  xlab("Samples") + ylab("read counts")
readCounts_plot

# 4- Normalization by DESeq2

## By DESeq2

## -- WT --

In [ ]:
dds.g080 <- estimateSizeFactors(dds.g080)
sizeFactors(dds.g080)
normalized_counts_byDeseq2_g080 <- counts(dds.g080, normalized=TRUE)
head(normalized_counts_byDeseq2_g080)

# prepare files to save them later : by adding gene names
normalized_counts_byDeseq2_g080=as.data.frame(normalized_counts_byDeseq2_g080)
mapping_geneName =  getBM(attributes=c("ensembl_gene_id","external_gene_name"), mart=mart, values=normalized_counts_byDeseq2_g080, uniqueRows=TRUE, bmHeader = T)
normalized_counts_byDeseq2_g080$genes=row.names(normalized_counts_byDeseq2_g080)
normalized_counts_byDeseq2_g080 = merge(normalized_counts_byDeseq2_g080, mapping_geneName, by.x='genes', by.y="Gene stable ID", all.x=TRUE, all.y=FALSE)


## -- DPLD --

In [ ]:
dds.dpld <- estimateSizeFactors(dds.dpld)
sizeFactors(dds.dpld)
normalized_counts_byDeseq2_dpld <- counts(dds.dpld, normalized=TRUE)
head(normalized_counts_byDeseq2_dpld)

# prepare files to save them later : by adding gene names
normalized_counts_byDeseq2_dpld=as.data.frame(normalized_counts_byDeseq2_dpld)
mapping_geneName =  getBM(attributes=c("ensembl_gene_id","external_gene_name"), mart=mart, values=normalized_counts_byDeseq2_dpld, uniqueRows=TRUE, bmHeader = T)
normalized_counts_byDeseq2_dpld$genes=row.names(normalized_counts_byDeseq2_dpld)
normalized_counts_byDeseq2_dpld = merge(normalized_counts_byDeseq2_dpld, mapping_geneName, by.x='genes', by.y="Gene stable ID", all.x=TRUE, all.y=FALSE)

# 5- Transformation and quality control

## -- WT --

In [ ]:
#rlog transformation
rld.g080 <- rlog( dds.g080 )
assay(rld.g080)[ 1:3,]

options(repr.plot.width=10, repr.plot.height=10)


#plot : before transformation (just a simple log to help visualization
plot( log2( 1+counts(dds.g080, normalized=TRUE)[, 1:2] ), col="#00000020", pch=20, cex=0.3, main="non-transformed data", cex.main=3, cex.axis=2 )
#plot : after transformation 
plot( assay(rld.g080)[, 3:4], col="#00000020", pch=20, cex=0.3, main="rlog-transformed data", cex.main=3, cex.axis=2)


In [ ]:

# Distance matrice : assess overall similarity between samples
sampleDists_rld.g080 <- dist(t(assay(rld.g080)))
sampleDistMatrix_rld.g080 <- as.matrix( sampleDists_rld.g080 )
rownames(sampleDistMatrix_rld.g080) <- dds.g080$condition
colnames(sampleDistMatrix_rld.g080) <- colnames(rld.g080)  
colours = colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
pheatmap_distMatrice.g080=pheatmap(sampleDistMatrix_rld.g080,
         clustering_distance_rows=sampleDists_rld.g080,
         clustering_distance_cols=sampleDists_rld.g080,
         col=colours, main="G080 : Distance matrice after rlog transformation",     
cellheight=80, cellwidth = 80)

pheatmap_distMatrice.g080

## -- DPLD --

In [ ]:
#rlog transformation
rld.dpld <- rlog( dds.dpld )
assay(rld.dpld)[ 1:3,]

options(repr.plot.width=10, repr.plot.height=10)


#plot : before transformation (just a simple log to help visualization
plot( log2( 1+counts(dds.dpld, normalized=TRUE)[, 1:2] ), col="#00000020", pch=20, cex=0.3, main="non-transformed data", cex.main=3, cex.axis=2 )
#plot : after transformation 
plot( assay(rld.dpld)[, 3:4], col="#00000020", pch=20, cex=0.3, main="rlog-transformed data", cex.main=3, cex.axis=2)


In [ ]:

# Distance matrice : assess overall similarity between samples
sampleDists_rld.dpld <- dist(t(assay(rld.dpld)))
sampleDistMatrix_rld.dpld <- as.matrix( sampleDists_rld.dpld )
rownames(sampleDistMatrix_rld.dpld) <- dds.dpld$condition
colnames(sampleDistMatrix_rld.dpld) <- colnames(rld.dpld)  
colours = colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)
pheatmap_distMatrice.dpld=pheatmap(sampleDistMatrix_rld.dpld,
         clustering_distance_rows=sampleDists_rld.dpld,
         clustering_distance_cols=sampleDists_rld.dpld,
         col=colours, main="DPLD : Distance matrice after rlog transformation",     
cellheight=80, cellwidth = 80)

pheatmap_distMatrice.dpld

# 6- Running DESeq2

## -- WT --

In [ ]:
dds.g080 <- DESeq(dds.g080)
res.g080 <- results(dds.g080, alpha=0.05, contrast=c("condition", "synap","input"), pAdjustMethod="BH")
res.g080
summary(res.g080)

In [ ]:
#converting to dataframe
results.g080 = as.data.frame(res.g080)

## -- DPLD --

In [ ]:
dds.dpld <- DESeq(dds.dpld)
res.dpld <- results(dds.dpld, alpha=0.05, contrast=c("condition", "synap","input"), pAdjustMethod="BH")
res.dpld
summary(res.dpld)

In [ ]:
#converting to dataframe
results.dpld = as.data.frame(res.dpld)

# 7- Data analysis

## -- WT --

### plot 2 : VolcanoPlot

In [ ]:
#let's display and annotate genes that have significant LFC greater or equal to +0.85 / lower or equal to -0.85.
VolPlot.g080 = results.g080 %>% 
  mutate(
    Expression = case_when(log2FoldChange >= 0.85 & padj < 0.05 ~ "Up-regulated in synaptosome",
                           log2FoldChange <= -0.85 & padj < 0.05 ~ "Up-regulated in other fractions",
                           TRUE ~ "Unchanged")
    )


In [ ]:
# adding ENSEMBL gene names to the table
mapping_geneNames <-  getBM(attributes=c("ensembl_gene_id","external_gene_name", "transcript_biotype"), mart=mart, values=VolPlot.g080, uniqueRows=TRUE, bmHeader = T)
VolPlot.g080 = merge(VolPlot.g080,mapping_geneNames, by.x='row.names', by.y='Gene stable ID', all.x=TRUE, all.y=FALSE)
rownames(VolPlot.g080)=VolPlot.g080$Row.names
VolPlot.g080$Row.names=NULL
head(VolPlot.g080)

In [ ]:
# genes with significant LFC >= 0.85 are enriched in fractions enriched in synaptosomes.
# genes with significalt LFC <= -0.85 are enriched in the input franctions
top <- 20
top_genes <- bind_rows(
  VolPlot.g080 %>% 
    filter(Expression == 'Up-regulated in synaptosome') %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    head(top),
  VolPlot.g080 %>% 
    filter(Expression == 'Up-regulated in other fractions') %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    head(top)
)

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
VolcanoPlot.g080=ggplot(VolPlot.g080, aes(log2FoldChange, -log(padj,10))) +
  geom_point(aes(color = Expression), size = 5) +
  scale_color_manual(values = c("gray50","dodgerblue3",  "firebrick3")) +
  guides(colour = guide_legend(override.aes = list(size=20)))+
  geom_label_repel(data = top_genes,
                   mapping = aes(log2FoldChange, -log(padj,10), label = `Gene name`),
                   size = 5, face='bold')+
  theme_minimal()+
  theme(axis.text.x = element_text(angle=10, size = 10, face='bold'),
          axis.text.y = element_text(size = 10, face='bold'),
        axis.title = element_text(size =10, face='bold'),
         plot.title= element_text(hjust = 0.5, size = 20, face='bold'),
        legend.text = element_text(size = 20, face='bold'),
        legend.title = element_text(size = 20, face='bold'),
        legend.key.size = unit(3,"line"),
    legend.position = c(.1, .95),
    legend.justification = c("left", "top"),
    legend.box.just = "left",legend.margin = margin(3, 3, 3, 3))
VolcanoPlot.g080

### plot 3 : MAplot

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)

res_up <- results.g080 %>% 
             filter(log2FoldChange >= 0.85 & padj<0.05)

res_down <- results.g080 %>% 
             filter(log2FoldChange <= -0.85 & padj<0.05)

maplot_nolabel1 <- ggplot(results.g080, aes(log2(baseMean), log2FoldChange)) + 
        geom_point(alpha=0.3) +
        geom_point(data=res_up,
            aes(log2(baseMean), log2FoldChange),color="red2", cex=1.8)+
        geom_point(data=res_down,
            aes(log2(baseMean), log2FoldChange),color="cornflowerblue", cex=1.8) + 
        geom_hline(yintercept=0.85, linetype="dashed", color = "red")+ 
        geom_hline(yintercept=-0.85, linetype="dashed", color = "red")+
        ggtitle("G080 MAplot : synaptosome vs input")+
        theme(plot.title = element_text(color = "black", 
                                    size = 20,
                                    face = 2,
                                     hjust=0.5))+ 
        annotate("text", x = c(15,15), y = c(4,-4), 
        label = c(dim(res_up)[1], dim(res_down)[1]) , color=c("red2","cornflowerblue"), 
        size=7 , angle=0, fontface="bold") + theme_minimal()+
  scale_y_continuous(breaks = c(-3, -2, -1, 0, 1, 2, 3, 4, 5, 6))

maplot_nolabel1

## -- DPLD --

### plot 2 : VolcanoPlot

In [ ]:
#let's display and annotate genes that have significant LFC greater or equal to +0.85 / lower or equal to -0.85.
VolPlot.dpld = results.dpld %>% 
  mutate(
    Expression = case_when(log2FoldChange >= 0.85 & padj < 0.05 ~ "Up-regulated in synaptosome",
                           log2FoldChange <= -0.85 & padj < 0.05 ~ "Up-regulated in other fractions",
                           TRUE ~ "Unchanged")
    )


In [ ]:
# adding ENSEMBL gene names to the table
mapping_geneNames <-  getBM(attributes=c("ensembl_gene_id","external_gene_name", "transcript_biotype"), mart=mart, values=VolPlot.dpld, uniqueRows=TRUE, bmHeader = T)
VolPlot.dpld = merge(VolPlot.dpld,mapping_geneNames, by.x='row.names', by.y='Gene stable ID', all.x=TRUE, all.y=FALSE)
rownames(VolPlot.dpld)=VolPlot.dpld$Row.names
VolPlot.dpld$Row.names=NULL
head(VolPlot.dpld)

In [ ]:
# genes with significant LFC >= 0.85 are enriched in fractions enriched in synaptosomes.
# genes with significalt LFC <= -0.85 are enriched in the input franctions
top <- 20
top_genes <- bind_rows(
  VolPlot.dpld %>% 
    filter(Expression == 'Up-regulated in synaptosome') %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    head(top),
  VolPlot.dpld %>% 
    filter(Expression == 'Up-regulated in other fractions') %>% 
    arrange(padj, desc(abs(log2FoldChange))) %>% 
    head(top)
)

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
VolcanoPlot.dpld=ggplot(VolPlot.dpld, aes(log2FoldChange, -log(padj,10))) +
  geom_point(aes(color = Expression), size = 5) +
  scale_color_manual(values = c("gray50","dodgerblue3",  "firebrick3")) +
  guides(colour = guide_legend(override.aes = list(size=20)))+
  geom_label_repel(data = top_genes,
                   mapping = aes(log2FoldChange, -log(padj,10), label = `Gene name`),
                   size = 5, face='bold')+
  theme_minimal()+
  theme(axis.text.x = element_text(angle=10, size = 10, face='bold'),
          axis.text.y = element_text(size = 10, face='bold'),
        axis.title = element_text(size =10, face='bold'),
         plot.title= element_text(hjust = 0.5, size = 20, face='bold'),
        legend.text = element_text(size = 20, face='bold'),
        legend.title = element_text(size = 20, face='bold'),
        legend.key.size = unit(3,"line"),
    legend.position = c(.1, .95),
    legend.justification = c("left", "top"),
    legend.box.just = "left",legend.margin = margin(3, 3, 3, 3))
VolcanoPlot.dpld

### plot 3 : MAplot

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)

res_up <- results.dpld %>% 
             filter(log2FoldChange >= 0.85 & padj<0.05)

res_down <- results.dpld %>% 
             filter(log2FoldChange <= -0.85 & padj<0.05)

maplot.dPLD <- ggplot(results.dpld, aes(log2(baseMean), log2FoldChange)) + 
        geom_point(alpha=0.3) +
        geom_point(data=res_up,
            aes(log2(baseMean), log2FoldChange),color="red2", cex=1.8)+
        geom_point(data=res_down,
            aes(log2(baseMean), log2FoldChange),color="cornflowerblue", cex=1.8) + 
        geom_hline(yintercept=0.85, linetype="dashed", color = "red")+ 
        geom_hline(yintercept=-0.85, linetype="dashed", color = "red")+
        ggtitle("DPLD MAplot : synaptosome vs input")+
        theme(plot.title = element_text(color = "black", 
                                    size = 20,
                                    face = 2,
                                     hjust=0.5))+ 
        annotate("text", x = c(15,15), y = c(4,-4), 
        label = c(dim(res_up)[1], dim(res_down)[1]) , color=c("red2","cornflowerblue"), 
        size=7 , angle=0, fontface="bold") + theme_minimal()+
  scale_y_continuous(breaks = c(-3, -2, -1, 0, 1, 2, 3, 4, 5, 6))

maplot.dPLD